<img src="images/trace.png" style="float:right;width:100px;"> 

# Dillute trace elements - partitioning concept

In [ ]:
options(repr.plot.width=5.5, repr.plot.height=4.5)   # Size for the Jupyter plot (inches)
options(jupyter.plot_mimetypes = "image/png")        # Change mimetype to PNG

# Using image/svg+xml
# May not work in some browsers (but is optimal for Chrome, Firefox...)
options(repr.plot.width=5, repr.plot.height=5)
options(jupyter.plot_mimetypes = "image/svg+xml") 

### 14.2: Fractional crystallization (direct trc)
<img src="images/exercise-icon.png" style="float:left;width:40px;height:40px;">
The following file shows the REE concentrations in a tonalitic magma (ppm), as well as the partition coefficients for crystallizing mineral phases: 

<div style="text-align: right"> 
<img src="images/floppy.png" style="float:right;width:30px;height:30px;">
ttg_a.data </div>

A differentiated magma is generated by 30% fractional crystallization of a parental magma (H34); the cumulate consists of plagioclase, hornblende and ilmenite.
* Calculate the differentiated magma composition for a cumulate made up of 49 % plagioclase, 49 % hornblende, and 2 % ilmenite.
* By means of the *GCDkit* function `spider`, plot chondrite-normalized REE patterns (using normalization values after Boynton 1984).
<div style="text-align: right"> 
<img src="images/floppy.png" style="float:right;width:30px;height:30px;">
boynton.data </div>

In [ ]:
x <- read.table("data/ttg_a.data",sep="\t")
x <- as.matrix(x)               # transform dataframe to matrix
print(x)

In [ ]:
c0 <- x[,1]                     # parental magma composition
kd <- x[,-1]                    # table of distribution coefficients
fc <- 0.3                       # degree of fractionation
ff <- 1-fc                      # fraction of the melt left 
m1 <- c(0.49,0.49,0.02)         # mineral props in cumulate (1)

In [ ]:
dd1 <- kd%*%m1                  # bulk distrib. coeff. [Eq.(10.4)]

cl1 <- c0*ff^(dd1-1)            # melt composition, Rayleigh equation [Eq. (11.5)]

cs1 <- dd1*cl1                  # instantaneous solid [Eq. (11.8)]
csavg1 <- c0*(1-ff^dd1)/(1-ff)  # average solid [Eq. (11.9)]

result <- cbind(c0,dd1,cs1,csavg1,cl1)
colnames(result) <- c("C0","D1","CS1","CSavg1","CL1")

print(round(cbind(x,result),2))

In [ ]:
# Read the chondritic REE abundances (Boynton 1984) 
x <- read.table("data/boynton.data",sep=",")
chondrite <- as.numeric(x) # conversion to numeric vector
names(chondrite) <- names(x) 
print(chondrite)

In [ ]:
# Append empty rows for elements not modelled
ii <- !names(chondrite)%in%rownames(result)   # determine the missing REE
names.ii <- names(chondrite)[ii]
missing <- matrix(NA,nrow=length(names.ii),ncol=2,dimnames=list(names.ii,c("C0","CL1")))
result <- rbind(result[,c("C0","CL1")],missing) 
result <- result[names(chondrite),]
print(result)

In [ ]:
# Normalization by chondritic abundances
y <- result/chondrite
print(t(y),3)

# Plot
plot(y[,1],type="o",log="y",axes=FALSE,xlab="",ylab="REE/chondrite",ylim=c(0.1,1000),pch=15,col="black")
axis(1,1:length(chondrite),labels=names(chondrite),cex.axis=0.75)
axis(2)
points(y[,2],type="o",pch=1,col="red")
abline(h=(10^(-1:3)),lty="dashed")
box()

### 15.1: Fractional crystallization (reversed Ex. 14.2)

<img src="images/exercise-icon.png" style="float:left;width:40px;height:40px;">
The following file contains trace-element compositions of two tonalites, a primitive WR1 and a differentiated WR2, together with corresponding partition coefficients for the relevant rock-forming minerals.


<div style="text-align: right"> 
<img src="images/floppy.png" style="float:right;width:30px;height:30px;">
ttg_a2.data </div>

Let’s assume that `WR1` and `WR2` represent pure compositions of a tonalitic melt differentiating by Rayleigh-type fractional crystallization.
* Using the least-square method, estimate the modal composition of the cumulate and the degree of fractional crystallization.

In [ ]:
x <- read.table("data/ttg_a2.data",sep="\t")
x <- data.matrix(x)
print(x)

In [ ]:
dmat <- x[,-(1:2)]-1    # partitioning coeffs - 1 [Eq. (12.3)]
cv <- log(x[,2]/x[,1])  # log ratios of differentiated/parental magma comp. [Eq. (12.4)]

In [ ]:
ee <- lsfit(dmat,cv,intercept=FALSE)
mm <- ee$coeff
ff <- exp(sum(mm))      # fraction of melt remaining [Eq.(13.7)]

cat(round((1-ff)*100,1),"% fractional crystallization ","\n")
m <- mm/log(ff)*100     # mineral percentages [Eq. (13.8)]
print(round(m,1))

**References**

Boynton WV (1984) Cosmochemistry of the rare earth elements: meteorite studies. 
In: Henderson P (eds) Rare Earth Element Geochemistry. Elsevier, Amsterdam, pp 63-114